# Capstone Project - The Battle of Neighbourhoods
Opening a New Vegetarian/ Vegan Restaurant in Chennai 

### 1. Importing the required libraries

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
!conda install bs4 --yes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.6.2~ --> pkgs/main::ca-certificates-2020.6.24-0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2020.6.20-py36h9~ --> pkgs/main::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_1 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be UPDATED:

  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-

### 2. DataFrame Creation

In [45]:
from bs4 import BeautifulSoup # library to parse HTML and XML documents
import urllib.request

In [70]:
url="https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Chennai";
data = urllib.request.urlopen(url).read()
soup = BeautifulSoup(data, 'html.parser')
neighborhood = []

In [71]:
for row in soup.find_all("div", class_="mw-parser-output")[0].find_all("ul")[1].findAll("li"):
    neighborhood.append(row.text)
for row in soup.find_all("div", class_="mw-parser-output")[0].find_all("ul")[2].findAll("li"):
    neighborhood.append(row.text)
for row in soup.find_all("div", class_="mw-parser-output")[0].find_all("ul")[3].findAll("li"):
    neighborhood.append(row.text)
print(neighborhood)

['Red Hills', 'Royapuram', 'Korukkupet', 'Vyasarpadi', 'Tondiarpet', 'Tiruvottiyur', 'Ennore', 'Minjur', 'Old Washermenpet', 'Madhavaram', 'Manali New Town', 'Naravarikuppam', 'Sowcarpet', 'Puzhal', 'Moolakadai', 'Central', 'Kodungaiyur', 'Madhavaram Milk Colony', 'Mathur MMDA', 'Kolathur', "Parry's Corner", 'Purasawalkam', 'Perambur', 'Manali', 'Vallalar Nagar', 'New Washermenpet', 'Mannadi', 'George Town', 'Basin Bridge', 'Park Town', 'Periamet', 'Choolai', 'Vepery', 'Pattalam', 'Pulinanthope', 'M.K.B. Nagar', 'Selavoyal', 'Manjambakkam', 'Ponniammanmedu', 'Sembiam', 'T.V.K. Nagar', 'ICF Colony', 'Lakshmipuram', 'Oragadam', 'Ayapakkam', 'Pattravakkam', 'Thirumullaivoyal north', 'Avadi', 'Kathivakkam', 'Kathirvedu', 'Erukanchery', 'Broadway', 'Jamalia', 'Kallikuppam', 'Pattabiram', 'Kosapet', 'Villivakkam', 'Porur', 'manapakkam', 'Anna Nagar', 'Aminjikarai', 'Ambattur', 'Defence Colony', 'Padi', 'Ayappakkam', 'Korattur', 'Mogappair', 'Arumbakkam', 'Avadi', 'Pudur', 'Maduravoyal', 'Koy

In [72]:
# create a new DataFrame from the list
df = pd.DataFrame(neighborhood,columns=["Neighborhood"])
df.head()

,Neighborhood
0,Red Hills
1,Royapuram
2,Korukkupet
3,Vyasarpadi
4,Tondiarpet


In [73]:
df.shape #size of the data

(149, 1)

### 3. Get the lattitude and longitude

In [19]:
# define a function to get coordinates
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 17.4MB/s ta 0:00:01
     |████████████████████████████████| 829kB 25.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [74]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [75]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords


[[13.195430000000044, 80.18431000000004],
 [13.11394000000007, 80.29420000000005],
 [13.116800000000069, 80.27726000000007],
 [13.117780000000039, 80.25168000000008],
 [13.129230000000064, 80.28955000000008],
 [13.14178000000004, 80.29692000000006],
 [13.226280000000031, 80.32599000000005],
 [13.278720000000021, 80.26175000000006],
 [13.109690000000057, 80.28101000000004],
 [13.150430000000028, 80.23417000000006],
 [13.203710000000058, 80.26854000000003],
 [13.191070000000025, 80.18348000000003],
 [13.091000000000065, 80.28343000000007],
 [13.159460000000024, 80.20718000000005],
 [13.12838000000005, 80.24086000000005],
 [28.660000000000025, 77.21279000000004],
 [13.136630000000025, 80.24479000000008],
 [13.157520000000034, 80.24283000000008],
 [13.172150000000045, 80.24741000000006],
 [13.131830000000036, 80.19928000000004],
 [13.088670144430012, 80.29039072938303],
 [13.08525000000003, 80.25329000000005],
 [13.122470000000021, 80.23569000000003],
 [13.168640000000039, 80.2722400000000

In [76]:
# Dataframe for storing the latitude and longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [77]:
# Merging the tables
df[['Latitude','Longitude']] = df_coords[['Latitude','Longitude']]
df.head()

,Neighborhood,Latitude,Longitude
0,Red Hills,13.19543,80.18431
1,Royapuram,13.11394,80.29420
2,Korukkupet,13.11680,80.27726
3,Vyasarpadi,13.11778,80.25168
4,Tondiarpet,13.12923,80.28955


### 4. Create a map with neighborhoods

In [78]:
address = 'Chennai, India'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai are 13.0801721, 80.2838331.


In [79]:
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chennai)  
    
map_chennai

### 5. Use Foursquare to explore the neighborhood

In [1]:
CLIENT_ID = 'Enter client_id' # your Foursquare ID
CLIENT_SECRET = 'Enter client_secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Enter client_id
CLIENT_SECRET:Enter client_secret


#### Exploring the data

In [98]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [99]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Red Hills,13.19543,80.18431,Hotel Balaji Bavan,13.193716,80.185292,Indian Restaurant
1,Red Hills,13.19543,80.18431,Pvr Cinemas Skls Galaxy Mall,13.191586,80.186100,Movie Theater
2,Red Hills,13.19543,80.18431,Skls Galaxy Mall,13.191500,80.186038,Shopping Mall
3,Red Hills,13.19543,80.18431,Selvaraj Hotel,13.199747,80.178082,Indian Restaurant
4,Red Hills,13.19543,80.18431,Universal Gym,13.190710,80.177946,Gym


In [101]:
#size of the venues dataframe
print(venues_df.size)

28336


In [102]:
#count of each neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
,28,28,28,28,28,28
Adyar,83,83,83,83,83,83
Alandur,49,49,49,49,49,49
Alapakkam,22,22,22,22,22,22
Alwarthirunagar,30,30,30,30,30,30
Ambattur,12,12,12,12,12,12
Aminjikarai,94,94,94,94,94,94
Anna Nagar,100,100,100,100,100,100
Annanur,5,5,5,5,5,5


In [103]:
# get the unique category
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 189 uniques categories.


In [104]:
venues_df['VenueCategory'].unique()

array(['Indian Restaurant', 'Movie Theater', 'Shopping Mall', 'Gym',
       'Hotel', 'Motorcycle Shop', 'Convenience Store',
       'Vegetarian / Vegan Restaurant', 'Train Station',
       'Harbor / Marina', 'Platform', 'Bus Station', 'Multiplex',
       'Coffee Shop', 'Pizza Place', 'Snack Place', 'Beach',
       'Indie Movie Theater', 'Art Gallery', 'Scenic Lookout', 'Park',
       'Bakery', 'IT Services', 'Department Store', 'Lake', 'Restaurant',
       'Video Store', 'Museum', 'Sandwich Place', 'Nightclub',
       'Bookstore', 'Market', 'Soccer Stadium', 'Outlet Store',
       'Historic Site', 'Fast Food Restaurant', 'Campground', 'Pharmacy',
       'Boutique', 'Food & Drink Shop', 'Dessert Shop', 'Hardware Store',
       'Food', 'Paper / Office Supplies Store', 'Motel', 'Hostel',
       'Ice Cream Shop', 'Moving Target', 'ATM', 'Food Truck',
       'Athletics & Sports', 'Intersection', 'Italian Restaurant',
       'Farmers Market', 'Seafood Restaurant', 'Thai Restaurant', 'Café',


### 6. Analyse each neighborhood

In [106]:
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(4048, 190)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,Hospital,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music School,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Red Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Red Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Red Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Red Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [107]:
grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped

(146, 190)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,Hospital,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music School,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.035714,0.000000,0.00000,0.035714,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.035714,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.0,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.107143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.035714,0.000000,0.000000

In [108]:
grouped.head()

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,Hospital,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music School,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071429,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.035714,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.035714,0.000000,0.0,0.0,0.000000,0.142857,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.035714,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.035714,0.0,0.0,0.0,0.0,0.0,0.035714,0.000000,0.000000,0.035714,0.0,0.0,0.000000,0.0,0.000000,0.107143,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.035714,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.071429,0.0,0.035714,0.035714,0.0,0.000000,0.0,0.0,0.0
1,Adyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024096,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.024096,0.0,0.012048,0.0,0.0,0.012048,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.060241,0.000000,0.060241,0.

### 7. Seperate the vegetarian Restaurant and analayse

In [110]:
len(grouped[grouped["Vegetarian / Vegan Restaurant"] > 0])

57

In [112]:
restaurant_df = grouped[["Neighborhoods","Vegetarian / Vegan Restaurant"]]
restaurant_df.head()


,Neighborhoods,Vegetarian / Vegan Restaurant
0,,0.035714
1,Adyar,0.012048
2,Alandur,0.000000
3,Alapakkam,0.090909
4,Alwarthirunagar,0.100000


### 8. Cluster Neighborhoods

In [113]:
# set number of clusters
kclusters = 3

clustering = restaurant_df.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 0, 0, 2, 0, 0, 2, 0], dtype=int32)

In [114]:
# create a new dataframe 
new_df = restaurant_df.copy()

# add clustering labels
new_df["Cluster Labels"] = kmeans.labels_

new_df.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
new_df.head()


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels
0,,0.035714,0
1,Adyar,0.012048,2
2,Alandur,0.000000,2
3,Alapakkam,0.090909,0
4,Alwarthirunagar,0.100000,0


In [115]:
# merge the table with lat and log
new_df = new_df.join(df.set_index("Neighborhood"), on="Neighborhood")

print(new_df.shape)
new_df.head()

(147, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,,0.035714,0,13.08362,80.28252
1,Adyar,0.012048,2,13.00305,80.25193
2,Alandur,0.000000,2,13.00013,80.20060
3,Alapakkam,0.090909,0,13.04610,80.16499
4,Alwarthirunagar,0.100000,0,13.05055,80.18397


In [116]:
new_df.sort_values(["Cluster Labels"], inplace=True)
new_df

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,,0.035714,0,13.083620,80.282520
30,ICF Colony,0.038462,0,13.094620,80.218390
122,Thirumangalam,0.093023,0,13.082850,80.196990
120,Taramani,0.037736,0,12.982990,80.241250
37,Karambakkam,0.050000,0,13.036960,80.156450
116,Sowcarpet,0.038462,0,13.091000,80.283430
81,Nolambur,0.047619,0,13.076260,80.171690
111,Saligramam,0.046875,0,13.054800,80.201130
110,Saidapet,0.037037,0,13.020260,80.221310
49,Koyambedu,0.068966,0,13.066050,80.197230


### 9. Visualize the cluster

In [117]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighborhood'], new_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [119]:
new_df.loc[new_df['Cluster Labels'] == 0]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,,0.035714,0,13.08362,80.282520
30,ICF Colony,0.038462,0,13.09462,80.218390
122,Thirumangalam,0.093023,0,13.08285,80.196990
120,Taramani,0.037736,0,12.98299,80.241250
37,Karambakkam,0.050000,0,13.03696,80.156450
116,Sowcarpet,0.038462,0,13.09100,80.283430
81,Nolambur,0.047619,0,13.07626,80.171690
111,Saligramam,0.046875,0,13.05480,80.201130
110,Saidapet,0.037037,0,13.02026,80.221310
49,Koyambedu,0.068966,0,13.06605,80.197230


In [121]:
new_df.loc[new_df['Cluster Labels'] == 1]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
109,Royapuram,0.200000,1,13.11394,80.29420
45,Korukkupet,0.200000,1,13.11680,80.27726
61,Mangadu,0.250000,1,13.03060,80.10903
84,Padi,0.136364,1,13.09756,80.18679


In [122]:
new_df.loc[new_df['Cluster Labels'] == 2]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
114,Sembiam,0.000000,2,13.123210,80.242650
96,Perambur,0.000000,2,13.122470,80.235690
131,Tiruvottiyur,0.000000,2,13.141780,80.296920
95,Peerkankaranai,0.000000,2,12.912240,80.098950
94,Pazhavanthangal,0.000000,2,12.986810,80.186730
93,Pattravakkam,0.000000,2,13.111644,80.156137
92,Pattalam,0.000000,2,13.097000,80.259220
91,Pattabiram,0.000000,2,13.123330,80.059440
125,Thirumullaivoyal north,0.000000,2,13.127500,80.131640
126,Thirunindravur,0.000000,2,13.125130,80.042010


### Observation:
From the above analysis, we clearly understand that the cluster 1 has the most number of Vegetarian / Vegan Restaurants and moderate in cluster 0. Whereas in cluster 2 we can see that it contains only a few or no restaurants. It will be a high chance for us to get successed if we open Restaurant in and around the cluster 2 areas. In cluster 1, there may be heavy competition to survive in our business. 